In [18]:
import re
import os
import glob
import pandas as pd

from ast import literal_eval
from langdetect import detect
from itertools import combinations

import requests
import json
import time
from fuzzywuzzy.process import dedupe as fuzzy_dedupe

In [136]:
df = pd.read_csv('C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\ibsen_network\\data\\csv\\per_dacy_large.csv', converters={'dacy_large': literal_eval})

In [137]:
df

,files,text,dacy_large
0,BREV_B1844-1871ht_18670308FH,NaN,[]
1,BREV_B1844-1871ht_B18260306NTB,NaN,[]
2,BREV_B1844-1871ht_B18440520PL,"Du maa virkelig undskylde, at jeg først nu bes...","[Hedevall, Reimann, Reimann, Johan, Carl Aamod..."
3,BREV_B1844-1871ht_B18441006HSte,"Tilgiv at jeg ikke før har besvaret Dit Brev, ...","[Reimann, Mdm Reimann, Mdm R, Reimanns, Reiman..."
4,BREV_B1844-1871ht_B18450801AWE,\nJomfru M: Wahl hilses venskabeligst fra,[M: Wahl]
...,...,...,...
2444,BREV_B1890-1905ht_BudatNN_Hjerteligste,\nHjerteligste ønsker!Tør desværre ikke selv p...,[]
2445,BREV_B1890-1905ht_BudatNN_Med_udtrykket,\n\nMed udtrykket af min sympati for komitéens...,[]
2446,BREV_B1890-1905ht_BudatNN_Tallene,\nTallene må utvilsomt være skrevne af mig sel...,[]
2447,BREV_B1890-1905ht_BudatNN_Wenn_Sie,\n\n\n Wenn Sie keine andere Verwendung für Ih...,[]


In [138]:
df.dropna(subset = ["text"], inplace=True)

df['lang'] = df['text'].apply(detect)

In [139]:
df['lang'].unique()

array(['da', 'no', 'de', 'fr', 'en', 'it', 'nl', 'sv', 'vi', 'sw', 'af',
       'hu', 'id'], dtype=object)

In [140]:
df = df[(df["lang"] == 'da') | (df["lang"] == 'no') | (df["lang"] == 'sv')]

In [141]:
df = df[df['dacy_large'].map(lambda d: len(d)) > 0]

In [142]:
path = (r"C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\ibsen_network\\data\\")
df.to_csv(os.path.join(r'clean_dacy_large.csv'), encoding = 'utf-8', index = False)

In [143]:
df = pd.read_csv('C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\ibsen_network\\data\\csv\\clean_dacy_large.csv', converters={'dacy_large': literal_eval})

In [144]:
df

,files,text,dacy_large,lang
0,BREV_B1844-1871ht_B18440520PL,"Du maa virkelig undskylde, at jeg først nu bes...","[Hedevall, Reimann, Reimann, Johan, Carl Aamod...",da
1,BREV_B1844-1871ht_B18441006HSte,"Tilgiv at jeg ikke før har besvaret Dit Brev, ...","[Reimann, Mdm Reimann, Mdm R, Reimanns, Reiman...",da
2,BREV_B1844-1871ht_B18450801AWE,\nJomfru M: Wahl hilses venskabeligst fra,[M: Wahl],da
3,BREV_B1844-1871ht_B18461207JCP,Opfordret af Deres Velbaarenhed til at erklære...,"[Else Sophie Jensdatter Birkedalen, Hans Jacob...",da
4,BREV_B1844-1871ht_B18491015OS,Dit sidste Brev har i dobbelt Henseende glædet...,"[Olaf Tr, C.]",da
...,...,...,...,...
1271,BREV_B1890-1905ht_Budat1903VVe1,\nDr. Henrik Ibsen overlader gjerne «Tilskuere...,[Henrik Ibsen],da
1272,BREV_B1890-1905ht_Budat1903VVe2,\nDr. Henrik Ibsen tillader sig at forespørge ...,[Henrik Ibsen],da
1273,BREV_B1890-1905ht_BudatGET,\nDr. Henrik Ibsen sender sin Tak – for Aviser...,[Henrik Ibsen],da
1274,BREV_B1890-1905ht_BudatIAa_onsker,\nDr. Henrik Ibsen ønsker Theaterchef Bjørnson...,"[Henrik Ibsen, Theaterchef Bjørnson]",da


In [145]:
df = df[['text','dacy_large']]

In [146]:
data = df.to_dict('index')

In [147]:
def remove_dupes(article):
    contains_dupes = list(data[key]['dacy_large'])
    deduped = fuzzy_dedupe(contains_dupes)
    return deduped

removing duplicates in each letter

In [148]:
for key in data:
     # makes sure that the letter actually has text (defensive coding)
    if data[key]['text'] != '':
        people = remove_dupes(str(data[key]['dacy_large']))
        data[key]['people'] = people

Creating the adjacency matrix by making each entity a key and other entities in the same letter values of that key.

In [149]:
entities = {}

for key in data:
    people = data[key]['people']
    
    doc_ents = []
    for person in people:
            doc_ents.append(person)
    
    for ent in doc_ents:
        try:
            entities[ent].extend([doc for doc in doc_ents if doc != ent])
        except:
            entities[ent] = [doc for doc in doc_ents if doc != ent]

In [150]:
df = pd.DataFrame([entities])
df = df.transpose()

In [151]:
df.index.name = 'source'
df.reset_index(inplace=True)


In [152]:
df = df.rename(columns={0: 'target'})
df = df.explode('target')

df = df.dropna(subset = ['target'])

In [153]:
mask = (df['target'].str.len() > 3) & (df['source'].str.len() > 3)
df = df.loc[mask]

In [154]:
df

,source,target
0,Hedevall,Reimann
0,Hedevall,Johan
0,Hedevall,Carl Aamodt
1,Reimann,Hedevall
1,Reimann,Johan
...,...,...
1891,Laura Fitinghoff,Henrik Ibsen
1893,Wang,Peer Gynt
1894,Helga,Sigurds
1895,Hr. Skuespiller Garmann,Henrik Ibsen


In [156]:
df.to_csv('data_links.csv', index = True)